In [1]:
import random
import time
import pickle
import sys
import json
import re
import string
import collections
import codecs
from io import BytesIO

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib as mpl

import nltk
from nltk.corpus import stopwords
from nltk.tag.sequential import ClassifierBasedTagger
#from nltk.stem.wordnet import WordNetLemmatizer
#from nltk.tokenize import word_tokenize

from IPython.display import display, Image
from IPython.core.interactiveshell import InteractiveShell

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_colwidth', 120)

### Classifying the questions

In [2]:
frage_complete = pd.read_pickle('./frage.pickle')

In [3]:
frage_complete[['id', 'fragebogen_num', 'frage_num', 'original_frage', 'kurzfrage',]].head()

,id,fragebogen_num,frage_num,original_frage,kurzfrage
0,1,1,A1,"Kopf: Kopf, Haupt; auch scherzh./übertr.","Kopf, Haupt; auch scherzh./übertr."
1,2,1,A2,"Kopf: Kopf/Haupt (in urspr. Bed.) in Vergl./Ra. (Kopf stehn, der Kopf möchte einem zerspringen)",Kopf/Haupt (urspr.Bed.) in Vergl./Ra.*
2,3,1,A3,"Kopf: Kopf/Haupt (übertr.) in Vergl./Ra. (das ist ein feiner/offener Kopf, jem. den Kopf verdrehen)",Kopf/Haupt (übertr.) in Vergl./Ra.*
3,4,1,A4,"Kopf: schöner, ebenmäßig gebauter Kopf (Christuskopf, Tituskopf)","schöner, ebenmäßiger Kopf (Christus-, Titusk.)"
4,5,1,A4a,"Kopf: häßlicher, unebenmäßiger Kopf","häßlicher, unebenmäßiger Kopf"


In [4]:
frage_complete[['id', 'fragebogen_num', 'frage_num', 'original_frage', 'kurzfrage',]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16738 entries, 0 to 16743
Data columns (total 5 columns):
id                16738 non-null int64
fragebogen_num    16738 non-null int64
frage_num         16738 non-null object
original_frage    16738 non-null object
kurzfrage         16738 non-null object
dtypes: int64(2), object(3)
memory usage: 784.6+ KB


In [5]:
#frage_complete[frage_complete.original_frage.str.contains(pat = '\?')]['original_frage']
frage_complete[frage_complete.original_frage.str.contains(pat = 'wie |wer |was ', regex = True)]['original_frage']

116      Palmso.: Palmbuschen, Sachl.: woraus besteht er? (Kranewit, Wachholder, Gromelsträußlein, Waxlaub, Stechpalme, Schrä...
118      Palmso.: was geschieht m. d. Palmbuschen/-gerten? (Befestigg./Aufbewahrg. hinter Stubenkruzifixen, unter d. Dache, i...
124                                                       Palmso.: Palmesel; auch als SpottN?; Ra. (aufgeputzt wie ein Palmesel)
126      Palmso.: wird bei d. Palmeselfahrt e. Christbild in Holzplastik mitgeführt und wie heißt es?; wird Christus durch Pe...
129            Palmso.: wie heißt Woche vor d. Palmsonntag? haben Tage dieser Woche bes. Namen? (Palmfreitag, schmerzh. Freitag)
                                                                  ...                                                           
16466                                          Gewerbe: Goldarbeiter, Goldschmied; Ra. wie: Der macht's wie des Goldschmieds Bub
16472    Gewerbe: Gold; Ra. wie: Es ist nicht alles Gold, was glänzt; das ist nicht mit Gold zu b

#### Questions patterns

See:  
- [Document1](https://docs.google.com/document/d/1x_7nZmPZRva5LI_C8KgUzloSryaqwnaQsGlP7G-_VS0/edit#)  
- [Document2](https://docs.google.com/document/d/1Tpr4TvvIwqQLS7t9xQK0EPLErbTDuF5c4a0BdwsJ9wA/edit)  
- [Document3](https://docs.google.com/spreadsheets/d/1xxb-CjJwU_P44cCFr4II-7QFXaR5FB6ofcwMZRAhSmQ/edit#gid=1846077890)  

In [6]:
onomasiological_naming = ['Bez.', '-bez.', 'Bezeichng.','wie nennt man','was bezeichnet',
                          'wie heißen','wie heißt', 'Benennung', 'Benenng']
onomasiological_drawing = []
semasiological_meaning = ['Bedeutung', 'Bed.', 'was bedeutet']
agreement_yes_no = []
agreement_pronunciation = ['Ltg.','Lautung','Lautungen']
description_events = ['Beschr.','Beschreibg.','Beschreibung']
description_sayings = ['Ausdrücke']
illustration_drawing = []
morphological_compound_1 = ['Komp.']
morphological_compound_2 = ['Gw.']
morphological_compound_3 = ['Bw']
morphological_diminutive = ['Dem.']
morphological_plural = ['Plural','Pl.']
morphological_forms = []
morphological_derivatives = ['Abl.','Ableitung']
syntactic_constructions = ['Fügungen', 'Wendungen','Füg.']
syntactic_idioms = ['Sprichw.', 'Ra.']
phonological_pronunciation = ['Aussprache','Ausspr.', 'Ltg.']
metaphorical_conveyed_meanings = ['Übertr.', 'übertragener Bedeutung']
synonyms = ['Syn.', 'Synonym']
synonyms_contrast = ['Ggs.','Gegensatz'] 
synonyms_comparison = ['Vgl.','Vergl.','Vergleich']
cultural_beliefs = ['Aberglaube','Volksaberglaube','Volksglaube']
cultural_event = ['Bräuche', 'Brauch', 'dabei', 'Ausdrücke']
cultural_medicine = ['Volksheilmittel','Heilmittel','Blutreinigungsmittel','volkstüml','Gegenmittel']
cultural_prayer = ['Gebete']
cultural_songs = ['Lieder','Gesangs']
cultural_humor = ['Volkswitze','Mesnerwitze']
cultural_games_1 = ['Finger','Spiel','Kinderspiel','Laufspiele','spielen']
cultural_games_2 = ['Ausdrücke', 'Spiel']
cultural_food = ['zutat','speise','braten','Gebäck','Apfel','Kartoffel','Essen','Bier','trinken']
cultural_living_organisms = []
cultural_dances = ['Tänze','Tanz']

pattern_list = ['onomasiological_naming', 'onomasiological_drawing',
                'semasiological_meaning', 
                'agreement_yes_no', 'agreement_pronunciation', 
                'description_events', 'description_sayings', 
                'illustration_drawing', 
                'morphological_compound_1', 'morphological_compound_2', 'morphological_compound_3',
                'morphological_diminutive', 'morphological_plural', 'morphological_forms', 'morphological_derivatives', 
                'syntactic_constructions', 'syntactic_idioms',
                'phonological_pronunciation',
                'metaphorical_conveyed_meanings',
                'synonyms', 'synonyms_contrast', 'synonyms_comparison',
                'cultural_beliefs', 'cultural_event', 'cultural_medicine', 'cultural_prayer', 
                'cultural_songs', 'cultural_humor', 'cultural_games_1', 'cultural_games_2', 
                'cultural_food', 'cultural_living_organisms', 'cultural_dances'
               ]

Creating function to apply patterns

In [7]:
def test_type(entry):
    question_type = None
    for patterns in pattern_list:
        if eval(patterns):
            for pattern in eval(patterns):
                if pattern.lower() in entry.lower():
                    if question_type == None:
                        question_type = [patterns]
                    elif question_type == [patterns]:
                        break
                    else:
                        question_type.append(patterns)
                    break
    return question_type

Applying pattern:

In [8]:
frage_complete['frage_type'] = frage_complete.original_frage.apply(test_type)
print(f"{len(frage_complete['frage_type'].dropna())} questions were classified out of {len(frage_complete)}")
print('\nNumber of questions per number of classes:')
frage_complete['frage_type'].apply(lambda x:len(x) if x else None).value_counts(dropna=False)

6257 questions were classified out of 16738

Number of questions per number of classes:


NaN    10481
1.0     3587
2.0     1334
3.0      875
4.0      306
5.0      122
6.0       23
7.0        7
9.0        2
8.0        1
Name: frage_type, dtype: int64

Examining classified questions

In [9]:
frage_complete.loc[frage_complete['frage_type'].dropna().index, ['original_frage','frage_type']].head(20)

,original_frage,frage_type
0,"Kopf: Kopf, Haupt; auch scherzh./übertr.",[metaphorical_conveyed_meanings]
1,"Kopf: Kopf/Haupt (in urspr. Bed.) in Vergl./Ra. (Kopf stehn, der Kopf möchte einem zerspringen)","[semasiological_meaning, syntactic_idioms, synonyms_comparison]"
2,"Kopf: Kopf/Haupt (übertr.) in Vergl./Ra. (das ist ein feiner/offener Kopf, jem. den Kopf verdrehen)","[syntactic_idioms, metaphorical_conveyed_meanings, synonyms_comparison]"
20,"Kopf: Beule, Auswuchs am Kopf (Ra.)",[syntactic_idioms]
23,Kopf: sonstige Komp. mit Kopf/Schädel als Gw. (Weiberkopf),"[morphological_compound_1, morphological_compound_2]"
24,Kopf: sonstige Komp. mit Kopf/Schädel als Bw. (Kopfstück),"[morphological_compound_1, morphological_compound_3]"
26,"Totenkopf: Bräuche, die sich auf den Totenkopf beziehen (Waschen, Einwickeln, Färben desselben vor dem Wiedereingrab...",[cultural_event]
28,Totenkopf: Aberglaube,[cultural_beliefs]
30,"Kopf/Hirn: Hirn (nicht in Bed. ""Stirn"")",[semasiological_meaning]
31,Kopf: Komp. mit Hirn (hirnrissig usw.),[morphological_compound_1]


Examining unclassified questions

In [15]:
#frage_complete.style.set_properties(subset=['original_frage'], **{'width': '400px'})
frage_complete.loc[frage_complete['frage_type'].isnull(), ['original_frage']].head(50)

,original_frage
3,"Kopf: schöner, ebenmäßig gebauter Kopf (Christuskopf, Tituskopf)"
4,"Kopf: häßlicher, unebenmäßiger Kopf"
5,Kopf: großer Kopf; großkopfig
6,Kopf: Wasserkopf
7,Kopf: kleiner Kopf
8,Kopf: oben zugespitzter Kopf
9,Kopf: langer Kopf
10,"Kopf: kurzer, platter Kopf"
11,Kopf: runder Kopf
12,Kopf: breiter Kopf


In [11]:
frage_complete[['original_frage','frage_type']].to_pickle('./questions_class.pickle')